In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [4]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, 

In [5]:
len(final_documents)

164

In [6]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

C:\Users\hp\AppData\Local\Temp\ipykernel_26452\4110012675.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings=HuggingFaceBgeEmbeddings(
c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your 

In [7]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-0.07903485 -0.01134113 -0.02312097  0.02844461  0.05053344  0.05317826
 -0.01907787  0.03456026 -0.10211368 -0.02915702  0.0852426   0.05650727
 -0.02545439 -0.0330849  -0.00635735  0.04090864 -0.00628108  0.00356744
 -0.03854129  0.03667685 -0.04289803  0.03425252 -0.03116899 -0.03793729
  0.01728391  0.01214924  0.00653119  0.01463565 -0.05529054 -0.15320712
  0.00730845  0.03202944 -0.04701132 -0.01595974  0.0187445   0.02642936
 -0.02306378  0.08438035  0.04182485  0.05278177 -0.03057602  0.01564262
 -0.01689074  0.00529409 -0.02417436  0.00412995 -0.01889937 -0.00150625
 -0.00836945 -0.03390065  0.03515961 -0.00553131  0.04910938  0.05971856
  0.05615963 -0.05105155  0.01475136 -0.01849959 -0.03284641  0.03576624
  0.04947704 -0.00938883 -0.26202118  0.0975033   0.01715692  0.0478139
 -0.00556317 -0.00298307 -0.02207355 -0.04463669 -0.05760482  0.04815878
 -0.05522206  0.01635333  0.03299246  0.02147079  0.01296219  0.01462309
  0.02174952 -0.00202999  0.02099538  0.03353847 -0.

In [8]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [9]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022ABFBF2900> search_kwargs={'k': 3}


In [12]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HuggingFace_API_KEY")

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [18]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
load_dotenv()

client = InferenceClient(
    model="HuggingFaceH4/zephyr-7b-beta",  # or any other supported model
    token=os.getenv("HuggingFace_API_KEY")
)

prompt = "What is health insurance coverage?"
response = client.text_generation(
    prompt=prompt,
    max_new_tokens=300,
    temperature=0.1
)

print(response)



Health insurance coverage is a type of insurance that helps pay for medical expenses. It can cover a wide range of services, from routine check-ups and preventive care to major medical events like hospitalization and surgery. Health insurance can be provided by an employer, purchased individually, or obtained through a government program like Medicare or Medicaid.

Why is health insurance coverage important?

Health insurance coverage is important because it provides financial protection against unexpected medical expenses. Without insurance, a major medical event like a serious illness or injury could result in overwhelming debt. Health insurance coverage can also help individuals access preventive care and manage chronic conditions, which can lead to better overall health and lower healthcare costs in the long run.

What are some common types of health insurance coverage?

1. Preferred Provider Organization (PPO) - This type of plan allows individuals to choose their own healthcare

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class. -- Not running this
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [21]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [22]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [23]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [24]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [25]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

AttributeError: 'InferenceClient' object has no attribute 'post'

In [27]:
# ---------------------------
# 🔹 Setup & Token Handling
# ---------------------------
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the .env file
load_dotenv()

# Get your Hugging Face token from the environment
hf_token = os.getenv("HuggingFace_API_KEY")

# Ensure token is loaded
if not hf_token:
    raise ValueError("Hugging Face token not found. Make sure HF_TOKEN is set in your .env file.")

# ---------------------------
# 🔹 Load a gated LLM model
# ---------------------------
model_name = "google/flan-t5-small"  # Or any gated model you have access to

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)

# ---------------------------
# 🔹 Run inference
# ---------------------------
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Explain black holes in simple terms."
output = text_generator(prompt, max_new_tokens=100)[0]['generated_text']

print("🔹 Output:\n", output)

c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\auto\tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In o

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of AriaTextConfig, BambaConfig, BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitNetConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, Cohere2Config, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, DeepseekV3Config, DiffLlamaConfig, ElectraConfig, Emu3Config, ErnieConfig, FalconConfig, FalconMambaConfig, FuyuConfig, GemmaConfig, Gemma2Config, Gemma3Config, Gemma3TextConfig, GitConfig, GlmConfig, Glm4Config, GotOcr2Config, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GraniteConfig, GraniteMoeConfig, GraniteMoeHybridConfig, GraniteMoeSharedConfig, HeliumConfig, JambaConfig, JetMoeConfig, LlamaConfig, Llama4Config, Llama4Tex